step 1

In [29]:
pip install findspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
import findspark
findspark.init()

# Set the SPARK_HOME environment variable
import os
os.environ["SPARK_HOME"] = "/path/to/your/spark/home/directory"


In [31]:
import pandas as pd
from pyspark.sql import SparkSession
import pyspark
from pyspark import SparkContext
from pyspark.sql.functions import col
from pyspark.sql.functions import desc
import string
import re
from pyspark.sql.functions import split   
import pyspark.sql.functions as sqf
from pyspark.sql.functions import col, expr, when
import findspark


In [26]:
# Importing "Amazon_Responded_Oct05.csv" dataframe
Amazon_dataframe = pd.read_csv("Amazon_Responded_Oct05.csv", sep=",", error_bad_lines=False)
Amazon_dataframe

<ipython-input-26-ca85d6543719>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  Amazon_dataframe = pd.read_csv("Amazon_Responded_Oct05.csv", sep=",", error_bad_lines=False)
b'Skipping line 209619: expected 25 fields, saw 26\nSkipping line 209620: expected 25 fields, saw 26\n'


,id_str,tweet_created_at,user_screen_name,user_id_str,user_statuses_count,user_favourites_count,user_protected,user_listed_count,user_following,user_description,...,tweet_language,text_,favorite_count,favorited,in_reply_to_screen_name,in_reply_to_status_id_str,in_reply_to_user_id_str,retweet_count,retweeted,text
0,'793270689780203520',Tue Nov 01 01:57:25 +0000 2016,SeanEPanjab,143515471.0,51287,4079,False,74,False,Content marketer; Polyglot; Beard aficionado; ...,...,en,@AmazonHelp Can you please DM me? A product I ...,0,False,AmazonHelp,NaN,85741735.0,0,False,NaN
1,'793281386912354304',Tue Nov 01 02:39:55 +0000 2016,AmazonHelp,85741735.0,2225450,11366,False,796,False,We answer Amazon support questions 7 days a we...,...,en,"@SeanEPanjab I'm sorry, we're unable to DM you...",0,False,SeanEPanjab,7.932707e+17,143515471.0,0,False,NaN
2,'793501578766319616',Tue Nov 01 17:14:53 +0000 2016,SeanEPanjab,143515471.0,51287,4079,False,74,False,Content marketer; Polyglot; Beard aficionado; ...,...,en,@AmazonHelp It was purchased on https://t.co/g...,0,False,AmazonHelp,7.932814e+17,85741735.0,0,False,@AmazonHelp It was purchased on https://t.co/g...
3,'793501657346682880',Tue Nov 01 17:15:12 +0000 2016,SeanEPanjab,143515471.0,51287,4079,False,74,False,Content marketer; Polyglot; Beard aficionado; ...,...,en,"@AmazonHelp I am following you now, if it help...",0,False,AmazonHelp,7.932814e+17,85741735.0,0,False,NaN
4,'793502854459879424',Tue Nov 01 17:19:57 +0000 2016,AmazonHelp,85741735.0,2225450,11366,False,796,False,We answer Amazon support questions 7 days a we...,...,en,@SeanEPanjab Please give us a call/chat so we ...,0,False,SeanEPanjab,7.935016e+17,143515471.0,0,False,@SeanEPanjab Please give us a call/chat so we ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378127,'892535280460234753',Tue Aug 01 23:59:28 +0000 2017,AmazonHelp,85741735.0,2215790,11277,False,796,False,We answer Amazon support questions 7 days a we...,...,en,@Voltaireon Please send us the details here: h...,1,False,Voltaireon,8.925318e+17,278387899.0,0,False,@Voltaireon Please send us the details here: h...
378128,'892532821893533697',Tue Aug 01 23:49:41 +0000 2017,nat_says_,60186579.0,14857,229,False,5,False,nothing really going on here. just talking to ...,...,en,@AmazonHelp do you guys ship to APO addresses?,0,False,AmazonHelp,NaN,85741735.0,0,False,NaN
378129,'892533275968798720',Tue Aug 01 23:51:30 +0000 2017,AmazonHelp,85741735.0,2215792,11277,False,796,False,We answer Amazon support questions 7 days a we...,...,en,@nat_says_ We do! You can find more informatio...,0,False,nat_says_,8.925328e+17,60186579.0,0,False,NaN
378130,'892560509639241728',Wed Aug 02 01:39:43 +0000 2017,nat_says_,60186579.0,14857,229,False,5,False,nothing really going on here. just talking to ...,...,en,@AmazonHelp Thank you,0,False,AmazonHelp,8.925333e+17,85741735.0,0,False,NaN


In [32]:
# Converting amazon dataframe to str
amazon_df = Amazon_dataframe.astype(str) 

# Converting amazon dataframe to spark dataframe
findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()
amazon_spark_df = spark.createDataFrame(amazon_df)
amazon_spark_df.show()

+--------------------+--------------------+----------------+-----------+-------------------+---------------------+--------------+-----------------+--------------+--------------------+-------------+-------------+--------------------+------------------+--------------------+--------------+--------------------+--------------+---------+-----------------------+-------------------------+-----------------------+-------------+---------+--------------------+
|              id_str|    tweet_created_at|user_screen_name|user_id_str|user_statuses_count|user_favourites_count|user_protected|user_listed_count|user_following|    user_description|user_location|user_verified|user_followers_count|user_friends_count|     user_created_at|tweet_language|               text_|favorite_count|favorited|in_reply_to_screen_name|in_reply_to_status_id_str|in_reply_to_user_id_str|retweet_count|retweeted|                text|
+--------------------+--------------------+----------------+-----------+-------------------+--

In [33]:
# Extracting 3 useful columns from amazon spark dataframe
amazon_3_column = amazon_spark_df.select('tweet_created_at','user_screen_name','user_id_str')

# Converting the structure of date

date_str = split(amazon_3_column["tweet_created_at"], ' ')
amazon_3_column= amazon_3_column.withColumn('Month', date_str.getItem(1))
amazon_3_column = amazon_3_column.withColumn('Date', date_str.getItem(2))
amazon_3_column = amazon_3_column.withColumn('Year', date_str.getItem(5))

amazon_3_column = amazon_3_column.withColumn("tweet_created_at",sqf.concat(sqf.col("Month"), sqf.lit(" "), sqf.col("Date"),sqf.lit(" "), sqf.col("Year")))
amazon_3_column= amazon_3_column.select('tweet_created_at','user_screen_name','user_id_str')

# Leaving duplicate users aside
amazon_3_column = amazon_3_column.dropDuplicates()
amazon_3_column.show(10)
print(amazon_3_column.count())

+----------------+----------------+------------+
|tweet_created_at|user_screen_name| user_id_str|
+----------------+----------------+------------+
|     Nov 03 2016|   vivek_coool21| 305560776.0|
|     Nov 02 2016|    scorpio98012| 180331674.0|
|     Nov 02 2016|     amyirish101|4494066677.0|
|     Nov 04 2016|    ServantOnIce| 350308945.0|
|     Nov 04 2016|       chhuti_is|  67290915.0|
|     Nov 04 2016|  loveydhillon13|2854616179.0|
|     Nov 04 2016|      problesgil|  15250896.0|
|     Nov 04 2016| JustifiablyPale| 842317388.0|
|     Nov 05 2016|  RPG_Gamer_Luke|3528407837.0|
|     Nov 10 2016|     TanejaMudit| 507360196.0|
+----------------+----------------+------------+
only showing top 10 rows

93927


In [34]:
# Reporting the number of active users
daily_active_users = amazon_3_column.groupBy(amazon_3_column.user_screen_name,amazon_3_column.user_id_str).count()
daily_active_users = daily_active_users.filter("count >= 5")
daily_active_users = daily_active_users.select('user_id_str','user_screen_name')
daily_active_users.show(10)
print("Daily active users is", daily_active_users.count())

+--------------------+----------------+
|         user_id_str|user_screen_name|
+--------------------+----------------+
|         125675649.0|          txomon|
|          21041619.0|    RobynCobbold|
|         187257670.0|   mehulsthakker|
|          16543736.0|      manchicken|
|        2953709027.0| AmyUnitedStates|
|        3252507746.0|         KSean02|
|          49989431.0|        NOtoGMOs|
|          11702402.0|        trallyus|
|        1300002864.0|    matthewD2013|
|7.587445573159936...| Savvyy_Investor|
+--------------------+----------------+
only showing top 10 rows

Daily active users is 593


step 2

In [36]:
import os
if not os.path.exists('/content'):
    os.makedirs('/content')


In [40]:
daily_active_users.toPandas().to_csv('/content/daily_active_users_result.csv', index=False, header=True)


In [43]:
# Importing "experiment.txt" file as dataframe
exp_txt = pd.read_csv("/content/experiment.txt", sep=",", header=None,error_bad_lines=False)

# Convert exp_txt Dataframe to spark dataframe
exp_df = spark.createDataFrame(exp_txt)

# Renaming the column to user_id_exp
exp_df = exp_df.withColumnRenamed("0", "user_id_exp")

<ipython-input-43-3e9c28c8bf75>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  exp_txt = pd.read_csv("/content/experiment.txt", sep=",", header=None,error_bad_lines=False)


In [44]:
#left join should be used to join exp_df with daily_active_user as user_id from experiment.txt should remain the same. 
experiment_user = exp_df.join(daily_active_users,daily_active_users.user_id_str == exp_df.user_id_exp,"left")
experiment_user.show(10)

+-----------+-----------+----------------+
|user_id_exp|user_id_str|user_screen_name|
+-----------+-----------+----------------+
|   85741735| 85741735.0|      AmazonHelp|
|  143515471|       null|            null|
|  110354554|110354554.0| praveen_pandey_|
|   97424433|       null|            null|
|  902137872|902137872.0|     mybharatraj|
| 2706101936|       null|            null|
|   62364020|       null|            null|
|  106799492|       null|            null|
| 2908108256|       null|            null|
|   71457972|       null|            null|
+-----------+-----------+----------------+
only showing top 10 rows



In [45]:
# if user_id_str = user_id_exp, then active status should be 'yes', otherwise 'no' 
active_status = expr("""IF(user_id_str = user_id_exp, "Yes", "No")""")

# selecting 'user_id_str','whether_active'
experiment_user = experiment_user.withColumn("whether_active", active_status)
experiment_user = experiment_user.select('user_id_exp','whether_active')
experiment_user.show(10)
print("Experiment users", experiment_user.count(),".")

+-----------+--------------+
|user_id_exp|whether_active|
+-----------+--------------+
|   85741735|           Yes|
|  143515471|            No|
|  110354554|           Yes|
|   97424433|            No|
|  902137872|           Yes|
| 2706101936|            No|
|   62364020|            No|
|  106799492|            No|
| 2908108256|            No|
|   71457972|            No|
+-----------+--------------+
only showing top 10 rows

Experiment users 5000 .


In [46]:
# percentage of active users
experiment_user.groupBy("whether_active").count().show()
no = experiment_user.groupBy("whether_active").count().collect()[0]["count"]
yes = experiment_user.groupBy("whether_active").count().collect()[1]["count"]
percentage = yes/(no+yes)*100
print("% of active users", percentage,"%")

+--------------+-----+
|whether_active|count|
+--------------+-----+
|            No| 4885|
|           Yes|  115|
+--------------+-----+

% of active users 2.3 %


step 3

In [47]:
# importing final_experiment.csv as spark dataframe

final_experiment_df = pd.read_csv("/content/final_experiment (1).csv")
final_experiment_df = spark.createDataFrame(final_experiment_df)
final_experiment_df.show()
final_experiment_df.printSchema()

+-------------+----+--------------+----------------+
|  user_id_str|info|whether_active|user_screen_name|
+-------------+----+--------------+----------------+
|   6.236402E7|   F|           NaN|             NaN|
|2.706101936E9|   M|           NaN|             NaN|
|    5654472.0|   F|           NaN|             NaN|
| 1.45579921E8|   F|           NaN|             NaN|
|2.502172122E9|   M|           NaN|             NaN|
| 2.43716471E8|   F|           NaN|             NaN|
|2.610379644E9|   M|           NaN|             NaN|
| 1.23138418E8|   M|           NaN|             NaN|
| 2.57376764E8|   F|           NaN|             NaN|
| 2.69145593E8|   M|           NaN|             NaN|
| 3.70711133E8|   F|           NaN|             NaN|
|1.510968974E9|   F|           NaN|             NaN|
|3.526380922E9|   M|           NaN|             NaN|
| 1.63413904E8|   F|           NaN|             NaN|
|  1.6980347E7|   M|           NaN|             NaN|
|1.209614366E9|   M|           NaN|           

In [48]:
# counting instances in final_experiment_df
final_experiment_df.count()

4500

In [49]:
# Assign alias to different dataframe
final_exp = final_experiment_df.alias('fe')
final_exp.show(5)
exp_user = experiment_user.alias('e')
exp_user.show(5)
dly_act = daily_active_users.alias('a')
dly_act.show(5)

+-------------+----+--------------+----------------+
|  user_id_str|info|whether_active|user_screen_name|
+-------------+----+--------------+----------------+
|   6.236402E7|   F|           NaN|             NaN|
|2.706101936E9|   M|           NaN|             NaN|
|    5654472.0|   F|           NaN|             NaN|
| 1.45579921E8|   F|           NaN|             NaN|
|2.502172122E9|   M|           NaN|             NaN|
+-------------+----+--------------+----------------+
only showing top 5 rows

+-----------+--------------+
|user_id_exp|whether_active|
+-----------+--------------+
|   85741735|           Yes|
|  143515471|            No|
|  106799492|            No|
| 2908108256|            No|
|   71457972|            No|
+-----------+--------------+
only showing top 5 rows

+------------+----------------+
| user_id_str|user_screen_name|
+------------+----------------+
| 125675649.0|          txomon|
|  21041619.0|    RobynCobbold|
| 187257670.0|   mehulsthakker|
|  16543736.0|      

In [50]:
# using left join to join two dataframes by alias 
left_join_1 = exp_user.join(dly_act, dly_act.user_id_str == exp_user.user_id_exp,how='left')
left_join_1 = left_join_1.drop('user_id_str')

# leaving aside('whether_active','user_screen_name') from final_experiment
final_exp = final_exp.drop('whether_active','user_screen_name')

# Joining left_join_1 with the final_experiment alias using left join
left_join_2 = final_exp.join(left_join_1, final_exp.user_id_str == left_join_1.user_id_exp,how='left')
left_join_2 = left_join_2.drop('user_id_exp')
left_join_2.show(10)

+------------+----+--------------+----------------+
| user_id_str|info|whether_active|user_screen_name|
+------------+----+--------------+----------------+
|   5654472.0|   F|            No|            null|
| 1.7126306E7|   F|            No|            null|
| 1.9273402E7|   M|            No|            null|
| 4.1489388E7|   F|            No|            null|
|  6.236402E7|   F|            No|            null|
| 9.8177499E7|   F|            No|            null|
|1.23138418E8|   M|            No|            null|
|1.45579921E8|   F|            No|            null|
|1.46854424E8|   F|            No|            null|
|2.43716471E8|   F|            No|            null|
+------------+----+--------------+----------------+
only showing top 10 rows



In [51]:
# Insert "No record" in place of 'null'
final_experiment = left_join_2.na.fill("No record")
final_experiment.show(10)

+------------+----+--------------+----------------+
| user_id_str|info|whether_active|user_screen_name|
+------------+----+--------------+----------------+
|   5654472.0|   F|            No|       No record|
| 1.7126306E7|   F|            No|       No record|
| 1.9273402E7|   M|            No|       No record|
| 4.1489388E7|   F|            No|       No record|
|  6.236402E7|   F|            No|       No record|
| 9.8177499E7|   F|            No|       No record|
|1.23138418E8|   M|            No|       No record|
|1.45579921E8|   F|            No|       No record|
|1.46854424E8|   F|            No|       No record|
|2.43716471E8|   F|            No|       No record|
+------------+----+--------------+----------------+
only showing top 10 rows



In [52]:
print(final_experiment.count())

4500


In [53]:
# Save the result as csv file
final_experiment.toPandas().to_csv(r'/content/final_experiment (1).csv',index = False, header=True)

Rough Work





In [19]:
import pandas as pd

# Load experiment.txt file into a pandas dataframe
experiment_df = pd.read_csv('experiment.txt')

experiment.head(100
                )

[Row(143515471='85741735'),
 Row(143515471='71457972'),
 Row(143515471='2908108256'),
 Row(143515471='106799492'),
 Row(143515471='59156981'),
 Row(143515471='902137872'),
 Row(143515471='110354554'),
 Row(143515471='97424433'),
 Row(143515471='62364020'),
 Row(143515471='2706101936'),
 Row(143515471='5654472'),
 Row(143515471='145579921'),
 Row(143515471='2502172122'),
 Row(143515471='243716471'),
 Row(143515471='2610379644'),
 Row(143515471='123138418'),
 Row(143515471='257376764'),
 Row(143515471='269145593'),
 Row(143515471='370711133'),
 Row(143515471='1510968974'),
 Row(143515471='3526380922'),
 Row(143515471='163413904'),
 Row(143515471='16980347'),
 Row(143515471='1209614366'),
 Row(143515471='447433286'),
 Row(143515471='1970607968'),
 Row(143515471='3285473358'),
 Row(143515471='1112166661'),
 Row(143515471='422175328'),
 Row(143515471='60004557'),
 Row(143515471='2789430008'),
 Row(143515471='253242925'),
 Row(143515471='141214344'),
 Row(143515471='24787056'),
 Row(14351547